In [ ]:
import json
import os
from openai import OpenAI
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter  # For chunking
import fitz  # PyMuPDF


# --- OpenAI API 클라이언트 초기화 ---
# 환경 변수에서 API 키를 불러옵니다.
# 실제 사용 시에는 사용자 본인의 OPENAI_API_KEY를 환경 변수로 설정해야 합니다.
# 예시: os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY_HERE"
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY 환경 변수가 설정되지 않았습니다. API 키를 설정해주세요.")
client = OpenAI(api_key=api_key)
LLM_MODEL_FOR_REPORTING = "gpt-4o"  # 보고서 생성에 사용할 모델


# --- Helper Functions (개선된 부분) ---

def extract_text_with_page_info(pdf_path):
    """
    PDF 파일에서 텍스트를 페이지별로 추출하여 리스트로 반환합니다.
    각 리스트 항목은 한 페이지의 텍스트이며, 내부적으로 페이지 번호 정보가 유지됩니다.
    """
    document = fitz.open(pdf_path)
    page_texts = []
    for page_num in range(document.page_count):
        page = document.load_page(page_num)
        text = page.get_text()
        page_texts.append(text.strip())  # 각 페이지 텍스트를 리스트에 저장
    return page_texts, document.page_count  # 전체 텍스트를 하나의 문자열이 아닌 리스트로 반환


def extract_text_for_pages(page_texts_list, start_page_num, end_page_num):
    """
    페이지 텍스트 리스트에서 특정 페이지 범위의 텍스트를 추출합니다.
    """
    # PDF 페이지는 1부터 시작하므로 리스트 인덱스를 맞춥니다.
    start_idx = max(0, start_page_num - 1)
    end_idx = min(len(page_texts_list), end_page_num)  # end_page_num 포함

    if start_idx >= end_idx:
        return ""

    # 원하는 페이지 범위의 텍스트만 결합
    combined_text = "\n".join(page_texts_list[start_idx:end_idx])
    return combined_text


def get_toc_raw_text_from_full_text(page_texts_list, toc_page_numbers=[2, 3]):
    """
    전체 텍스트 리스트에서 지정된 목차 페이지들의 텍스트만 추출합니다.
    """
    toc_texts = []
    for page_num in toc_page_numbers:
        # extract_text_for_pages 함수는 1-based page numbers를 처리
        page_content = extract_text_for_pages(page_texts_list, page_num, page_num)
        if page_content:
            toc_texts.append(page_content)
        else:
            print(f"경고: 목차 페이지로 지정된 {page_num} 페이지에서 텍스트를 찾을 수 없습니다.")
    if not toc_texts:
        return None
    # 추출된 목차 텍스트를 LLM에 전달하기 전 간단히 정제
    raw_toc = "\n".join(toc_texts)
    # 불필요한 공백, 탭, 연속 줄바꿈 제거 (이후 LLM이 파싱하기 좋게)
    raw_toc = re.sub(r'\s+', ' ', raw_toc).strip()  # 여러 공백을 하나로, 줄바꿈을 공백으로
    raw_toc = re.sub(r'\.{2,}', '', raw_toc)  # 페이지 번호 앞의 점선 제거 (예: ...5 -> 5)
    raw_toc = re.sub(r'\n{2,}', '\n', raw_toc)  # 2개 이상의 줄바꿈을 1개로
    return raw_toc


def parse_toc_with_llm(toc_raw_text, client_instance, llm_model="gpt-4o"):
    """
    LLM을 사용하여 목차 원문 텍스트를 파싱하여 구조화된 목차 항목을 추출합니다.
    """
    system_prompt = """
    당신은 PDF 문서에서 추출된 목차(Table of Contents)의 원시 텍스트를 분석하는 전문가입니다.
    주어진 텍스트를 파싱하여 각 목차 항목의 제목, 페이지 번호, 그리고 해당 항목이 '요구사항' 관련 내용을 담고 있을 가능성을 분석하여 JSON 형태로 반환해야 합니다.
    JSON의 최상위 레벨은 "toc_entries"라는 키를 가진 객체여야 하고, 그 키의 값은 목차 항목 객체들의 리스트여야 합니다.
    **매우 중요: JSON 형식 외에 다른 어떠한 추가적인 설명이나 문장도 포함하지 마십시오.** 당신의 응답은 오직 JSON 객체여야 합니다.
    """
    user_prompt = f"""
    다음은 PDF에서 추출한 목차로 추정되는 텍스트입니다:

    --- 목차 원문 텍스트 시작 ---
    {toc_raw_text}
    --- 목차 원문 텍스트 끝 ---

    이 텍스트를 분석하여 JSON 객체를 반환해주세요. 이 객체는 "toc_entries"라는 키를 가져야 하며,
    이 키의 값은 각 목차 항목을 나타내는 객체들의 리스트여야 합니다.
    각 목차 항목 객체는 다음 키를 가져야 합니다:
    - "title": (문자열) 목차 항목의 전체 제목. 제목 앞의 번호(예: "1.", "II.", "가.", "1.1.", "제1장.")도 포함해주세요.
    - "page": (정수) 해당 항목의 시작 페이지 번호.
    - "is_requirement_related": (불리언) 제목이나 내용을 볼 때, 해당 항목이 '요구사항', '과업 범위', '제안 요청 상세', '기능 명세', '기술 요건', '현황', 'AS-IS', '재구축', '현재 시스템', '개선 방안' 등과 관련된 내용을 다룰 가능성이 높으면 true, 그렇지 않으면 false로 설정해주세요.

    예시 JSON 출력 형식:
    {{
      "toc_entries": [
        {{
          "title": "1. 사업 개요",
          "page": 5,
          "is_requirement_related": false
        }},
        {{
          "title": "III. 제안요청 내용",
          "page": 6,
          "is_requirement_related": true
        }},
        {{
          "title": "3. 상세 요구사항",
          "page": 11,
          "is_requirement_related": true
        }},
        {{
          "title": "* 보안 요구사항 별표",
          "page": 63,
          "is_requirement_related": true
        }},
        {{
          "title": "IV. 현행 시스템 분석",
          "page": 20,
          "is_requirement_related": true
        }},
        {{
          "title": "5. As-Is 시스템 현황",
          "page": 25,
          "is_requirement_related": true
        }}
      ]
    }}

    만약 주어진 텍스트가 유효한 목차로 보이지 않거나 항목을 전혀 파싱할 수 없다면,
    "toc_entries" 키의 값으로 빈 리스트 `[]`를 포함하는 JSON 객체를 반환해주세요. (예: {{"toc_entries": []}})
    """
    llm_response_content = ""
    try:
        response = client_instance.chat.completions.create(
            model=llm_model,
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.0  # 일관된 결과를 위해 0.0 유지
        )

        llm_response_content = response.choices[0].message.content
        parsed_data = json.loads(llm_response_content)

        extracted_list = []
        if isinstance(parsed_data, dict) and "toc_entries" in parsed_data and isinstance(parsed_data["toc_entries"], list):
            extracted_list = parsed_data["toc_entries"]
        else:
            print(f"LLM 응답이 예상된 'toc_entries' 리스트를 포함하는 객체 형식이 아닙니다. 응답: {llm_response_content[:200]}")
            return []

        valid_entries = []
        for entry in extracted_list:
            if isinstance(entry, dict) and 'title' in entry and 'page' in entry:
                try:
                    entry['page'] = int(entry['page'])
                    entry['is_requirement_related'] = bool(entry.get('is_requirement_related', False))
                    valid_entries.append(entry)
                except ValueError:
                    print(f"경고: 페이지 번호 '{entry.get('page')}'를 정수로 변환할 수 없습니다. 항목 건너뜀: {entry.get('title')}")
                    continue
            else:
                print(f"경고: 필수 키(title, page)가 누락된 항목입니다. 건너뜀: {entry}")

        return valid_entries

    except json.JSONDecodeError as e:
        print(f"LLM 목차 파싱 응답 JSON 파싱 오류: {e}. 응답 미리보기: {llm_response_content[:500]}...")
        return []
    except Exception as e:
        print(f"LLM 목차 파싱 중 예상치 못한 오류 발생: {e}")
        return []


def get_target_sections_from_llm_parsed_toc(parsed_toc_entries, total_pages):
    """
    LLM으로 파싱된 목차에서 As-Is 정보가 포함될 가능성이 있는 섹션 정보를 추출합니다.
    """
    target_sections = []
    # 페이지 번호 기준으로 정렬 (필수)
    sorted_toc = sorted(parsed_toc_entries, key=lambda x: x.get('page', 0))

    # 명시적으로 As-Is 정보를 포함할 가능성이 높은 키워드들 (소문자 비교를 위해 미리 소문자로 변환)
    keywords_to_find = [
        "현황 분석", "시스템 현황", "현행 시스템", "시스템 개요",
        "제안요청 내용", "상세 요구사항", "기능 요구사항", "비기능 요구사항",
        "보안 요구사항", "기술 현황", "아키텍처 현황",
        "현재 시스템", "as-is", "재구축", "목표시스템", "개선 방안",  # '개선 방안'에 현재 시스템의 문제점이 암시될 수 있음
        "사업 목표", "사업 내용"  # RFP의 초반부에 현황이 언급될 수 있음
    ]

    # As-Is 관련으로 식별된 모든 항목을 일차적으로 수집
    as_is_candidate_entries = []
    for entry in sorted_toc:
        entry_title_lower = entry.get('title', '').strip().lower()
        is_relevant_by_keyword = any(keyword in entry_title_lower for keyword in keywords_to_find)
        is_relevant_by_llm_flag = entry.get('is_requirement_related', False)

        if is_relevant_by_keyword or is_relevant_by_llm_flag:
            as_is_candidate_entries.append(entry)

    # 수집된 후보 항목들을 기반으로 실제 섹션 범위 결정 및 병합
    final_target_sections = []
    if not as_is_candidate_entries:  # 후보 섹션이 없으면 전체 문서를 대상으로
        print("경고: LLM 파싱 목차에서 주요 As-Is 관련 섹션을 찾지 못했습니다. 전체 문서를 대상으로 As-Is 분석을 시도합니다.")
        return [{'title': '전체 문서 (As-Is 섹션 식별 실패)', 'start_page': 1, 'end_page': total_pages}]

    current_start_page = as_is_candidate_entries[0].get('page', 1)
    current_end_page = current_start_page
    current_title = as_is_candidate_entries[0].get('title', 'AS-IS 관련 섹션')

    for i in range(len(as_is_candidate_entries)):
        entry = as_is_candidate_entries[i]
        entry_page = entry.get('page', 0)

        # 다음 섹션이 현재 처리 중인 범위에 바로 이어서 나오거나 겹칠 경우
        # 다음 섹션이 없거나, 다음 섹션의 페이지가 현재 섹션의 끝 페이지 + 1 보다 클 경우
        if i + 1 < len(as_is_candidate_entries):
            next_entry_page = as_is_candidate_entries[i + 1].get('page', total_pages + 1)
        else:
            next_entry_page = total_pages + 1  # 마지막 항목 이후는 문서 끝까지

        # 현재 항목의 끝 페이지는 다음 항목의 시작 페이지 -1 또는 문서 끝까지
        potential_end_page = min(total_pages, next_entry_page - 1)

        # 현재 entry_page가 current_end_page에 가깝거나, 현재 범위 내에 있다면 범위 확장
        if entry_page <= current_end_page + 1:  # 1페이지 차이까지는 연속으로 간주
            current_end_page = max(current_end_page, potential_end_page)
        else:  # 새로운 연속되지 않는 섹션 시작
            final_target_sections.append({
                'title': current_title,
                'start_page': current_start_page,
                'end_page': current_end_page
            })
            current_start_page = entry_page
            current_end_page = potential_end_page
            current_title = entry.get('title', 'AS-IS 관련 섹션')

    # 마지막 섹션 추가
    final_target_sections.append({
        'title': current_title,
        'start_page': current_start_page,
        'end_page': current_end_page
    })

    # 최종 결과 출력 및 정렬
    for section in final_target_sections:
        print(f"식별된 As-Is 관련 섹션: '{section['title']}' (페이지 {section['start_page']}-{section['end_page']})")

    return sorted(final_target_sections, key=lambda x: x['start_page'])


def summarize_chunk_for_as_is(chunk_text, client_instance, llm_model_name):
    """
    단일 청크에서 As-Is 보고서의 각 섹션에 해당하는 내용을 추출하고 요약하여 JSON으로 반환합니다.
    이때, 기능 목록은 동적으로 파악합니다.
    """
    extraction_system_prompt = """
    당신은 RFP 문서의 한 부분을 분석하여 **현재 시스템(AS-IS)의 현황과 특징에만 집중**하여 보고서의 각 섹션별로 정보를 추출하고 요약하는 전문가입니다.
    주어진 텍스트 청크에서 아래 JSON 구조에 맞춰 현재 시스템의 현황을 **최대한 상세하고 구체적으로** 추출하십시오.

    - **절대 추측하거나 없는 내용을 만들어내지 마십시오.** 오직 주어진 텍스트 청크에 **명시적/암시적으로 언급된 현재 상태(AS-IS) 정보만** 작성합니다.
    - 특히, "필요합니다", "개선 예정", "목표", "해야 한다"와 같은 **미래(To-Be) 지향적인 내용은 철저히 제외**하십시오.
    - "현재 이러이러한 문제가 있습니다", "현재 이러이러한 제약사항이 있습니다", "현재 이러이러한 기능만 지원합니다", "현재 이러이러한 방식입니다"와 같이 **현재 상태를 설명하는 부정적인 표현, 한계점, 결함, 또는 특정 방식에 대한 언급이 있다면, 이는 중요한 AS-IS 정보이므로 반드시 상세히 기술**하십시오.
    - 해당 섹션에 대한 AS-IS 정보가 텍스트 청크에 없으면, 해당 JSON 필드는 **"정보 없음"**으로 명확히 남겨 두십시오. 빈 문자열("")보다 "정보 없음"이 더 명확합니다.
    - 모든 추출 내용은 요약하거나 중요한 부분을 발췌하되, 가능한 한 원문의 의미를 살려 구체적으로 작성하십시오.
    - **'dynamic_functional_areas'에는 RFP에 언급된 현재 시스템의 모든 핵심 기능들을 기능별로 상세히 설명하십시오.** 각 기능의 **현재 운영 방식, 특징, 한계점** 등을 구체적으로 서술해야 합니다.
    """

    user_prompt_chunk = f"""
    다음 RFP 텍스트 청크에서 현재 시스템(As-Is) 관련 정보를 위 JSON 구조에 맞춰 추출하여 반환해 주십시오.

    --- 추출할 정보 구조 (JSON 형식) ---
    {{
        "overview": "현재 JBANK 시스템의 목적, 구축 배경, 시스템 구성, 재구축 목표 중 현행 시스템과 관련된 부분, 현재 거래 처리량(TPS), 향후 거래량 증가 예측 등 현행 시스템의 전반적인 특징을 상세히 요약합니다. '재구축을 통해 성능 향상 및 보안 강화'와 같은 목표 언급이 있더라도, 이것이 현재 시스템의 한계를 내포한다면 AS-IS 정보로 볼 수 있습니다. (예: 현재 시스템은 50 TPS를 처리하며, 특정 이벤트 시 성능 저하 우려가 있음)",
        "dynamic_functional_areas": {{
            "기능명1": "기능1의 현재 운영 방식, 특징, 한계점 등 상세 설명 (예: (O2O) 비대면 대출 연장 신청 프로세스 개발 적용, (O2O) 비대면 서류 작성 시스템 개발 적용, 신한은행 마이데이터 서비스(머니버스) 연동(웹뷰 방식) 등 RFP에 언급된 현재 구현된 기능 중심으로 상세히 작성)",
            "기능명2": "기능2에 대한 현재 상세 설명",
            "기능명N": "기능N에 대한 현재 상세 설명"
        }},
        "non_functional_aspects": {{
            "performance": "현재 시스템의 성능 관련 특징 (예: 현재 거래 처리량 50TPS, 향후 5년간 100TPS 증가 예상, 서비스의 성능 및 속도 보장 필요성 언급을 통해 현재의 잠재적 한계점 유추, 이벤트 시 사용자 급증에 따른 성능 저하 우려 등 현재 상황에 대한 언급)을 구체적인 수치와 함께 상세히 기술",
            "security": "현재 시스템의 보안 체계 (예: 로그 적재 시 비밀번호 마스킹 처리, 계정계 시스템과의 통신 시 암호화 방식 적용), 금융감독원 보안성 심의 가이드 준수 필요성 언급을 통해 현재 보안 수준 유추, 보안 취약점 예방 및 앱 접근성 인증마크 획득 방안 필요성 언급을 통해 현재의 부족한 부분을 AS-IS로 명시",
            "data": "현재 데이터 관리 방식 (예: 데이터 이행 계획 수립 여부, 데이터 매핑/초기 이행/변경 이행 포함 여부, 데이터 검증 및 정비 절차 유무, 데이터 오류 유형별 대응 방안 제시 여부)의 상세 현황을 기술",
            "ui_ux": "현재 모바일 뱅킹 앱의 사용자 인터페이스 및 사용자 경험에 대한 RFP의 언급 (예: 모바일 최적화 웹 개발 프레임워크 사용 권장이라는 문구를 통해 현재 최적화가 부족할 수 있음을 유추, 웹표준/웹접근성 자동 검증 기능 유무, UX 개선안 정의 및 사용자 검증 필요성 언급을 통해 현재 UX의 부족함 유추)을 상세히 설명",
            "stability": "시스템의 가용성 (예: 이중화 및 HA(Active-Active) 구성 원칙, DR 시스템은 싱글 구조로 제안되어 운영 환경과 동일한 시스템 구조와 소프트웨어 구성 갖춤), 서비스 오류 감지 및 원인 파악을 위한 거래 추적 구조 필요성 언급을 통해 현재 추적 기능의 부족함 유추 등 안정성 관련 현황을 구체적으로 설명",
            "constraints": "현재 시스템이 가진 기술적 (예: 개방형 구조의 Linux 기반, DB 서버는 Oracle RAC 구성), 운영적 한계 또는 외부 제약사항 (예: 프로젝트 진행 중 금융 관련 법규의 신설/변경, 감독당국의 지시사항 수용 필요)을 상세히 기술"
        }},
        "tech_architecture": {{
            "tech_stack": "현재 시스템이 사용하는 주요 기술 스택 (예: 운영체제 - 개방형 구조의 Linux 기반, 데이터베이스 - Oracle RAC 구성, 모바일 웹 개발 프레임워크 사용 권장(react.js, vue.js, spring boot)을 통해 현재 어떤 기술이 사용되거나 부족한지 유추)을 구체적으로 나열",
            "architecture": "현재 시스템의 전반적인 아키텍처 (예: 3 Tier(WEB-AP-DB) 구조, 주요 시스템 구간별 고가용성 확보, 컨텐츠 관리 방식(일괄 다운로드 방식 추정)) 및 구성 방식(다중화, 상호백업)을 상세히 설명",
            "integration_systems": "현재 연동하고 있는 주요 내부/외부 시스템 (예: 신한은행 마이데이터 서비스(머니버스) 연동(웹뷰 방식), 당행 통합모니터링(H/W, 어플리케이션, 네트워크 전구간 모니터링, 장애감지, 실시간 모니터링, 장애처리, 백업/복구 대응)과 연동) 현황 및 연동 방식을 구체적으로 설명"
        }}
    }}
    ---

    --- 텍스트 청크 시작 ---
    {chunk_text}
    --- 텍스트 청크 끝 ---
    """
    try:
        response = client_instance.chat.completions.create(
            model=llm_model_name,
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": extraction_system_prompt},
                {"role": "user", "content": user_prompt_chunk}
            ],
            temperature=0.0  # 일관된 결과를 위해 0.0 유지
        )
        extracted_info_str = response.choices[0].message.content
        extracted_info = json.loads(extracted_info_str)
        return extracted_info
    except json.JSONDecodeError as e:
        print(f"   경고: 청크 처리 중 JSON 파싱 오류: {e}. 응답 미리보기: {extracted_info_str[:200]}...")
        return None
    except Exception as e:
        print(f"   오류: 청크 처리 중 LLM API 호출 또는 처리 실패: {e}")
        return None

def consolidate_section_content(section_title, all_extracted_texts, client_instance, llm_model_name, is_dynamic_functional=False):
    """
    주어진 섹션 제목과 해당 섹션에 대해 모든 청크에서 추출된 텍스트들을 통합하여 상세 보고서 내용을 생성합니다.
    동적 기능 섹션의 경우 별도 처리가 필요합니다.
    (주의: 이 함수는 더 이상 마크다운 헤딩을 자체적으로 추가하지 않습니다. 내용은 일반 텍스트로 반환합니다.)
    """
    if not all_extracted_texts:
        # 섹션 제목 자체를 포함하지 않음. 호출하는 곳에서 헤딩을 추가할 것이기 때문
        return "RFP 텍스트에서 관련 정보를 충분히 찾을 수 없습니다."

    if is_dynamic_functional:
        combined_functions = {}
        for func_dict in all_extracted_texts:
            if isinstance(func_dict, dict):
                for func_name, func_desc in func_dict.items():
                    func_name = func_name.strip()
                    func_desc = func_desc.strip()
                    if func_name and func_desc not in ["정보 없음", ""]:
                        if func_name in combined_functions:
                            if func_desc and func_desc not in combined_functions[func_name]:
                                if combined_functions[func_name]:
                                    combined_functions[func_name] += f"\n- {func_desc}"
                                else:
                                    combined_functions[func_name] = func_desc
                        elif func_desc:
                            combined_functions[func_name] = func_desc

        if not combined_functions:
            return "RFP 텍스트에서 주요 기능을 찾을 수 없습니다."

        functions_list_for_llm = []
        for name, desc in combined_functions.items():
            # 여기서는 '### {name}'과 같이 기능별 서브-서브-서브 헤딩을 유지하여 기능 구분
            functions_list_for_llm.append(f"### {name}")
            functions_list_for_llm.append(desc if desc else "현재 상세 정보 없음.")
            functions_list_for_llm.append("")

        combined_texts_for_llm = "\n".join(functions_list_for_llm)
        if len(combined_texts_for_llm) > 40000:
            combined_texts_for_llm = combined_texts_for_llm[:40000] + "\n... (생략됨)"

        # ... (이전 코드와 동일한 LLM 호출 로직)
        section_consolidation_prompt = f"""
        당신은 RFP 문서의 '현황 분석(As-Is) 보고서' 중 '**{section_title}**' 섹션에 대한 전문 작성자입니다.
        다음은 RFP 문서의 여러 부분에서 추출된 현재 시스템의 주요 기능들에 대한 정보들입니다.
        이 정보들을 종합하고, 중복을 제거하며, 가장 상세하고 구체적인 내용만을 선별하여 Markdown 형식으로 **전문적이고 상세한 보고서 내용**을 작성해 주십시오.

        - **각 기능을 독립적인 서브 섹션(####)으로 나누어 현재 운영 방식, 특징, 한계점 등을 상세히 설명하십시오.**
        - **절대 추측하거나 없는 내용을 만들어내지 마십시오.** 오직 제공된 정보 내에서만 내용을 구성하십시오.
        - 내용이 불충분하더라도, 제공된 정보 내에서 최대한 상세하게 작성하십시오.
        - 제목을 따로 작성하지 않습니다.

        --- 추출된 '{section_title}' 관련 정보들 ---
        {combined_texts_for_llm}
        ---
        """
        try:
            response = client_instance.chat.completions.create(
                model=llm_model_name,
                messages=[
                    {"role": "system", "content": section_consolidation_prompt},
                ],
                temperature=0.1,
                max_tokens=2000
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"   오류: '{section_title}' 섹션 통합 생성 중 LLM API 호출 실패: {e}")
            return f"RFP 텍스트에서 주요 기능을 찾을 수 없습니다. (처리 오류: {e})"

    else:
        unique_texts = list(set([text.strip() for text in all_extracted_texts if text.strip()]))
        unique_texts = [text for text in unique_texts if text not in ["정보 없음", ""]]

        if not unique_texts:
            return "RFP 텍스트에서 관련 정보를 찾을 수 없습니다."

        combined_texts_for_llm = "\n- ".join(unique_texts)
        if len(combined_texts_for_llm) > 40000:
            combined_texts_for_llm = combined_texts_for_llm[:40000] + "\n... (생략됨)"

        # ... (이전 코드와 동일한 LLM 호출 로직)
        section_consolidation_prompt = f"""
        당신은 RFP 문서의 '현황 분석(As-Is) 보고서' 중 '**{section_title}**' 섹션에 대한 전문 작성자입니다.
        다음은 RFP 문서의 여러 부분에서 추출된 '{section_title}' 관련 정보들입니다.
        이 정보들을 종합하고, 중복을 제거하며, 가장 상세하고 정확한 내용만을 선별하여 Markdown 형식으로 **전문적이고 구체적인 보고서 내용**을 작성해 주십시오.

        - **절대 추측하거나 없는 내용을 만들어내지 마십시오.** 오직 제공된 정보 내에서만 내용을 구성하십시오.
        - 특히, "필요합니다", "개선 예정", "목표", "해야 한다"와 같은 **미래(To-Be) 지향적인 내용은 철저히 제외**하십시오.
        - "현재 이러이러한 문제가 있습니다", "현재 이러이러한 제약사항이 있습니다", "현재 이러이러한 기능만 지원합니다", "현재 이러이러한 방식입니다"와 같이 **현재 상태를 설명하는 부정적인 표현, 한계점, 결함, 또는 특정 방식에 대한 언급이 있다면, 이는 중요한 AS-IS 정보이므로 반드시 상세히 기술**하십시오.
        - 내용이 불충분하더라도, 제공된 정보 내에서 최대한 상세하게 작성하십시오.
        - 제목을 따로 작성하지 않습니다.

        --- 추출된 '{section_title}' 관련 정보들 ---
        {combined_texts_for_llm}
        ---
        """
        try:
            response = client_instance.chat.completions.create(
                model=llm_model_name,
                messages=[
                    {"role": "system", "content": section_consolidation_prompt},
                ],
                temperature=0.1,
                max_tokens=2000
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"   오류: '{section_title}' 섹션 통합 생성 중 LLM API 호출 실패: {e}")
            return f"RFP 텍스트에서 관련 정보를 찾을 수 없습니다. (처리 오류: {e})"

def generate_as_is_report_from_rfp_text(
    page_texts_list,
    total_pages,
    client_instance,
    llm_model_name,
    target_sections_for_analysis,
):
    """
    RFP 전체 텍스트(리스트 형태)를 청크로 나누고, 각 청크에서 As-Is 정보를 추출/요약하여
    최종 As-Is 분석 보고서를 생성합니다. 섹션별 구분을 제거하고, 하나의 통합된 보고서를 생성합니다.
    """
    print("\n[As-Is 보고서 생성] LLM을 사용하여 현황 분석 보고서를 생성합니다...")

    # AS-IS 분석 대상 섹션들의 텍스트만 추출하여 하나의 긴 텍스트로 결합
    all_as_is_relevant_text_parts = []
    for section_info in target_sections_for_analysis:
        section_text = extract_text_for_pages(
            page_texts_list, section_info["start_page"], section_info["end_page"]
        )
        if section_text:
            all_as_is_relevant_text_parts.append(section_text)
        else:
            print(
                f"      경고: '{section_info['title']}' 섹션 (페이지 {section_info['start_page']}-{section_info['end_page']})에서 텍스트를 추출하지 못했습니다."
            )

    if not all_as_is_relevant_text_parts:
        print("오류: As-Is 분석을 위한 텍스트를 전혀 추출하지 못했습니다. 보고서 생성을 중단합니다.")
        return "As-Is 보고서 생성이 실패했습니다: 분석할 텍스트가 없습니다."

    combined_rfp_text_for_as_is = "\n\n".join(all_as_is_relevant_text_parts)
    print(
        f"   As-Is 분석을 위한 RFP 텍스트 결합 완료. 총 길이: {len(combined_rfp_text_for_as_is)}자."
    )

    # 텍스트 청크 분할 설정 (청크 오버랩 500자)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=15000,  # 청크 크기를 15000으로 줄여 더 잘게 쪼갬
        chunk_overlap=500,  # 청크 간 오버랩
        length_function=len,
        separators=["\n\n", "\n", ". ", "? ", "! ", " ", ""],  # 페이지 마커는 이미 제거되었으므로 일반적인 구분자 사용
        keep_separator=False,
    )
    rfp_chunks = text_splitter.split_text(combined_rfp_text_for_as_is)
    print(f"   AS-IS 관련 텍스트를 총 {len(rfp_chunks)}개의 청크로 분할했습니다 (오버랩 포함).")

    # 모든 청크에서 추출된 정보를 저장할 구조
    all_extracted_data = {
        "overview": [],
        "dynamic_functional_areas": [],
        "non_functional_aspects": {
            "performance": [],
            "security": [],
            "data": [],
            "ui_ux": [],
            "stability": [],
            "constraints": [],
        },
        "tech_architecture": {
            "tech_stack": [],
            "architecture": [],
            "integration_systems": [],
        },
    }

    # 1단계: 각 청크에서 As-Is 관련 정보 추출 (JSON 형식)
    for i, chunk in enumerate(rfp_chunks):
        print(f"   청크 {i+1}/{len(rfp_chunks)}에서 정보 추출 중...")
        extracted_info = summarize_chunk_for_as_is(
            chunk, client_instance, llm_model_name
        )

        if extracted_info:
            # '정보 없음' 또는 빈 문자열이 아닌 경우에만 추가
            if extracted_info.get("overview") not in ["정보 없음", ""]:
                all_extracted_data["overview"].append(extracted_info["overview"])

            if extracted_info.get("dynamic_functional_areas"):
                all_extracted_data["dynamic_functional_areas"].append(
                    extracted_info["dynamic_functional_areas"]
                )

            for key, val in extracted_info.get("non_functional_aspects", {}).items():
                if val not in ["정보 없음", ""]:
                    all_extracted_data["non_functional_aspects"][key].append(val)

            for key, val in extracted_info.get("tech_architecture", {}).items():
                if val not in ["정보 없음", ""]:
                    all_extracted_data["tech_architecture"][key].append(val)

    print("\n[2단계: 추출된 정보 통합 및 보고서 생성]")
    final_as_is_report_parts = []

    # 2.1 개요 통합
    overview_content = consolidate_section_content(
        "시스템 개요", all_extracted_data["overview"], client_instance, llm_model_name
    )
    final_as_is_report_parts.append("## 1. 현행 시스템 개요\n")
    final_as_is_report_parts.append(overview_content)

    # 2.2 주요 기능 현황 통합 (Dynamic Functional Areas)
    functional_content = consolidate_section_content(
        "주요 기능 현황",
        all_extracted_data["dynamic_functional_areas"],
        client_instance,
        llm_model_name,
        is_dynamic_functional=True,
    )
    final_as_is_report_parts.append("\n\n## 2. 주요 기능 현황\n")
    final_as_is_report_parts.append(functional_content)

    # 2.3 비기능 요구사항 현황 통합
    final_as_is_report_parts.append("\n\n## 3. 비기능 요구사항 현황\n")
    non_functional_titles = {
        "performance": "가. 성능",
        "security": "나. 보안",
        "data": "다. 데이터",
        "ui_ux": "라. UI/UX",
        "stability": "마. 안정성",
        "constraints": "바. 제약사항",
    }
    for key, title in non_functional_titles.items():
        content = consolidate_section_content(
            title, # consolidate_section_content로 전달하는 title은 LLM 프롬프트에 사용
            all_extracted_data["non_functional_aspects"][key],
            client_instance,
            llm_model_name,
        )
        # 여기서 헤딩을 명확히 추가
        final_as_is_report_parts.append(f"\n### {title}\n{content}") # <-- 이 부분이 중요

    # 2.4 기술 아키텍처 현황 통합
    final_as_is_report_parts.append("\n\n## 4. 기술 아키텍처 현황\n")
    tech_arch_titles = {
        "tech_stack": "가. 기술 스택",
        "architecture": "나. 아키텍처",
        "integration_systems": "다. 연동 시스템",
    }
    for key, title in tech_arch_titles.items():
        content = consolidate_section_content(
            title, # consolidate_section_content로 전달하는 title은 LLM 프롬프트에 사용
            all_extracted_data["tech_architecture"][key],
            client_instance,
            llm_model_name,
        )
        # 여기서 헤딩을 명확히 추가
        final_as_is_report_parts.append(f"\n### {title}\n{content}") # <-- 이 부분이 중요

    final_report = "\n".join(final_as_is_report_parts)
    print("\n[As-Is 보고서 생성 완료]")
    return final_report


# --- 메인 실행 함수 (예시) ---
if __name__ == "__main__":
    # 이 부분은 실제 PDF 파일 경로로 대체해야 합니다.
    pdf_file_path = "docs/제주은행_RFP.pdf"  # 예시 파일명

    if not os.path.exists(pdf_file_path):
        print(f"오류: PDF 파일 '{pdf_file_path}'을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
    else:
        print(f"PDF 파일 '{pdf_file_path}'에서 텍스트를 추출 중...")
        page_texts, total_pages = extract_text_with_page_info(pdf_file_path)

        if not page_texts:
            print("PDF에서 텍스트를 추출하는 데 실패했습니다. 파일이 유효한지 확인해주세요.")
        else:
            print(f"총 {total_pages} 페이지의 텍스트를 추출했습니다.")

            # 1. 목차 페이지 텍스트 추출
            # RFP 문서의 실제 목차 페이지 번호를 확인하여 필요시 조정하세요.
            toc_raw_text = get_toc_raw_text_from_full_text(page_texts, toc_page_numbers=[2, 3])

            if toc_raw_text:
                print("\n[목차 파싱] LLM을 사용하여 목차를 파싱합니다...")
                parsed_toc = parse_toc_with_llm(toc_raw_text, client, LLM_MODEL_FOR_REPORTING)

                if parsed_toc:
                    print("\n[대상 섹션 식별] 파싱된 목차에서 As-Is 관련 섹션을 식별합니다...")
                    target_sections = get_target_sections_from_llm_parsed_toc(parsed_toc, total_pages)

                    if target_sections:
                        # 현황 분석 보고서 생성
                        as_is_report = generate_as_is_report_from_rfp_text(
                            page_texts, total_pages, client, LLM_MODEL_FOR_REPORTING, target_sections
                        )

                        # 결과 출력 또는 파일 저장
                        output_filename = "As_Is_Analysis_Report_Integrated.md"
                        with open(output_filename, "w", encoding="utf-8") as f:
                            f.write(as_is_report)
                        print(f"\n성공적으로 현황 분석 보고서를 '{output_filename}' 파일로 저장했습니다.")
                        print("\n--- 현황 분석 보고서 내용 미리보기 ---")
                        print(as_is_report[:2000]) # 보고서의 처음 2000자만 출력하여 미리보기
                        print("\n----------------------------------")
                    else:
                        print("As-Is 분석을 위한 목차 섹션을 식별할 수 없습니다. 보고서 생성을 건너뜜.")
                else:
                    print("목차 파싱에 실패했습니다. 보고서 생성을 건너뜜.")
            else:
                print("목차 원시 텍스트를 추출할 수 없습니다. 보고서 생성을 건너뛰고 전체 문서를 대상으로 시도할 수 있습니다.")
                # 목차 추출 실패 시 전체 문서를 대상으로 As-Is 분석 시도
                print("\n[목차 추출 실패] 전체 문서를 대상으로 As-Is 분석을 시도합니다...")
                target_sections_full_document = [{'title': '전체 문서', 'start_page': 1, 'end_page': total_pages}]
                as_is_report_full = generate_as_is_report_from_rfp_text(
                    page_texts, total_pages, client, LLM_MODEL_FOR_REPORTING, target_sections_full_document
                )
                output_filename_full = "As_Is_Analysis_Report_Full_Document.md"
                with open(output_filename_full, "w", encoding="utf-8") as f:
                    f.write(as_is_report_full)
                print(f"\n성공적으로 전체 문서를 기반으로 한 현황 분석 보고서를 '{output_filename_full}' 파일로 저장했습니다.")
                print("\n--- 현황 분석 보고서 내용 미리보기 (전체 문서 기반) ---")
                print(as_is_report_full[:2000]) # 보고서의 처음 2000자만 출력하여 미리보기
                print("\n----------------------------------")